# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [632]:
# импорт необходимых библиотек
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

# чтение исходного файла
client_data = pd.read_csv('/datasets/data.csv')

# Анализ общей информации о исходном файле (кол-во строк, столбцов, типы данных и т.п.)
client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### -----------------------------------------------------------------------------------------------------------------------------

Файл с данными имеет следующие характеристики:
* 21 525 строк
* 12 столбцов

Описание содержимого столбцов:
* 'children' — количество детей в семье (тип данных: int, кол-во записей - 21 525)
* 'days_employed' — общий трудовой стаж в днях (тип данных: float, кол-во записей - 19 351)
* 'dob_years' — возраст клиента в годах (тип данных: int, кол-во записей - 21 525)
* 'education' — уровень образования клиента (тип данных: str, кол-во записей - 21 525)
* 'education_id' — идентификатор уровня образования (тип данных: int, кол-во записей - 21 525)
* 'family_status' — семейное положение (тип данных: str, кол-во записей - 21 525)
* 'family_status_id' — идентификатор семейного положения (тип данных: int, кол-во записей - 21 525)
* 'gender' — пол клиента (тип данных: str, кол-во записей - 21 525)
* 'income_type' — тип занятости (тип данных: str, кол-во записей - 21 525)
* 'debt' — имел ли задолженность по возврату кредитов (тип данных: int, кол-во записей - 21 525)
* 'total_income' — ежемесячный доход (тип данных: float, кол-во записей - 19 351)
* 'purpose' — цель получения кредита (тип данных: str, кол-во записей - 21 525)

Пропуски, причем в равном количестве, присутствуют только в столбцах `days_employed` и `total_income`.

### -----------------------------------------------------------------------------------------------------------------------------

In [633]:
# Ознакомление со структурой данных
client_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### -----------------------------------------------------------------------------------------------------------------------------

В целом, названия столбцов достаточно информативны и стандартизированы, но, с целью большей удобочитаемости, стоит перименовать следующие столбцы:

* `days_employed` --> `seniority`
* `dob_years` --> `age`
* `income_type` --> `position`
* `total_income` --> `client_income`

### -----------------------------------------------------------------------------------------------------------------------------

In [634]:
# Переименовываем столбцы
client_data.rename(columns = {'days_employed':'seniority', 'dob_years':'age', 'income_type':'position', 'total_income':'client_income'}, inplace = True)

### -----------------------------------------------------------------------------------------------------------------------------

Результат:

### -----------------------------------------------------------------------------------------------------------------------------

In [635]:
client_data

,children,seniority,age,education,education_id,family_status,family_status_id,gender,position,debt,client_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### -----------------------------------------------------------------------------------------------------------------------------

Более внимательно пранализируем основные метрики столбцов с количественными показателями:

### -----------------------------------------------------------------------------------------------------------------------------

In [636]:
client_data.describe()

,children,seniority,age,education_id,family_status_id,debt,client_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### -----------------------------------------------------------------------------------------------------------------------------

Наблюдаются странные значения в ряде столбцов:

столбец `children`:
   * минимальное значение равное '-1'. Наиболее вероятное объяснение - ошибка при заполнении анкеты.   
   * максимальное значение равно '20'. Возможно, но стоить проверить "массовость" такого случая.
   
столбец `seniority`:
   * наблюдается массовое присутствие отрицательных значений. Возможное объяснение - отрицательными показателями характеризуют стаж людей уже вышедших на пенсию... Необходима проверка данной гипотезы.
   * среднее значение равно '63046.497661' дней (173 года). С данными явно что то не так... Необходимо проверить.

столбец `age`:
   * минимальное значение равно '0'. Маловато для заемщика... Необходимо разобраться.
   
### -----------------------------------------------------------------------------------------------------------------------------

### Вывод

В результате первичного анализа исходных данных, выявдены следующие проблемные моменты, требующие доп. проверки и/или решения:

- пропуски (причем в равном количестве) в `seniority` и `client_income`
- отрицательные значения в `children` и `seniority`
- значение '0' в `age`
- значение '20' в `children`
- неудобочитаемый формат данных (float) в `seniority` и `client_income`

В ходе дальнейшей работы, стоит провести более детальный анализ содержимого каждого из столбцов на предмет дополнительных аномалий (регистр и т.п.)
 

## Шаг 2. Предобработка данных

### Обработка пропусков

### -----------------------------------------------------------------------------------------------------------------------------

Еще раз проанализируем наличие пропусков:

### -----------------------------------------------------------------------------------------------------------------------------

In [637]:
client_data.isnull().sum()

children               0
seniority           2174
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
position               0
debt                   0
client_income       2174
purpose                0
dtype: int64

### -----------------------------------------------------------------------------------------------------------------------------

Наблюдаем одинаковое количество пропусков в `seniority` и `client_income`. Проверим в одинаковых ли строках встречаются пропуски.

### -----------------------------------------------------------------------------------------------------------------------------

In [638]:
client_data[(client_data['seniority'].isnull() == True) & (client_data['client_income'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
seniority           0 non-null float64
age                 2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
position            2174 non-null object
debt                2174 non-null int64
client_income       0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


### -----------------------------------------------------------------------------------------------------------------------------

Наше предположение подтвердилось - пропуски в `seniority` и `client_income` соответствуют друг другу. Возможно, что это технический сбой при фрмировании данных. Одобрение кредита таким клиентам влечет за собой значительные риски, а значит имеет смысл заполнить пропуски медианами по роду деятельности: пенсионеров - медианами пенсионеров, сотрудников - медианами сотрудников.

### -----------------------------------------------------------------------------------------------------------------------------

In [639]:
# Заменяем пропуски в 'seniority' на '0'
client_data['seniority'] = client_data['seniority'].fillna(0)

Заменим отсутствующие значения дохода медианными значениями отдельных, соответствующих, групп занятости (`position`)

Сначала построим таблицу с этими значениями:

In [640]:
income_median_by_position = pd.pivot_table(client_data, index = 'position', values = 'client_income', aggfunc = 'median')
income_median_by_position

,client_income
position,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


Создадим функцию для заполнения пустых значений дохода соответствующими медианными значениями групп занятости:

In [641]:
def mediana_for_none_income(row):
    return income_median_by_position.loc[row['position']]['client_income']

Заполним отстутствующие значения дохода с помощью вышесозданной функции

In [642]:
client_data.loc[client_data['client_income'].isnull(), 'client_income'] = client_data.apply(mediana_for_none_income, axis = 1)

Удостоверимся в отсутствии пустых значений в столбце `client_income`

In [643]:
client_data['client_income'].isnull().sum()

0

### -----------------------------------------------------------------------------------------------------------------------------

Удостоверимся, что пропусков больше нет.

### -----------------------------------------------------------------------------------------------------------------------------

In [644]:
client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
seniority           21525 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
position            21525 non-null object
debt                21525 non-null int64
client_income       21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропуски в `seniority` и `client_income` оказались взаимосвязаны и были заменены:

- в столбце `seniority` на нулевые значения, в связи с тем, что показатели стажа, в рамках стоящих перед нами задач, не понадобятся
- в столбце `client_income` на медианные значения дохода соответствующей группы занятости, во избежание искажений дальнейших вычислений.


В результате  - исходный файл не содержит пропусков.

### Замена типа данных

### -----------------------------------------------------------------------------------------------------------------------------

Имеет смысл, с целью повышения удобочитаемости таблицы изменить тип данных в столбце `client_income` с float на int, так как привычнее воспринимать доход в виде целочисленного значения. Для `seniority` оставим float, c целью более детальной информации о стаже (если преобразовать в int, многие значения отобразяться как '0').

### -----------------------------------------------------------------------------------------------------------------------------

In [645]:
# Меняем тип данных столбца 'client_income' на  int
client_data ['client_income'] =  client_data ['client_income'].astype('int')

#### Результат:

In [646]:
client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
seniority           21525 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
position            21525 non-null object
debt                21525 non-null int64
client_income       21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

### -----------------------------------------------------------------------------------------------------------------------------

Изменили тип данных в столбце `client_income` на  int, что повысило удобочитаемость данных.

### -----------------------------------------------------------------------------------------------------------------------------

## Обработка дубликатов

### -----------------------------------------------------------------------------------------------------------------------------

Оценим ситуацию с дублями.

### -----------------------------------------------------------------------------------------------------------------------------

In [647]:
# Считаем дубли
print('Количество дублей:', client_data.duplicated().sum())

Количество дублей: 54


### -----------------------------------------------------------------------------------------------------------------------------

Ознакомимся со строками-дубликатами

### -----------------------------------------------------------------------------------------------------------------------------

In [648]:
client_data[client_data.duplicated(keep = False)].sort_values(by = ['age'])

,children,seniority,age,education,education_id,family_status,family_status_id,gender,position,debt,client_income,purpose
19321,0,0.0,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
15892,0,0.0,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
18328,0,0.0,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
3452,0,0.0,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
8629,1,0.0,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
13639,0,0.0,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,автомобиль
3609,0,0.0,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,жилье
12389,0,0.0,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
5865,0,0.0,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью


### -----------------------------------------------------------------------------------------------------------------------------

Возможно, дубликаты могли появиться из за того, что выгрузка данных осуществлялась из разрозненных информационных систем  банка... Учитывая их относительно небольшое количество и невозможность получить комментарии относительно их возможной природы - целесообразно их просто удалить.

### -----------------------------------------------------------------------------------------------------------------------------

In [649]:
# Удаляем дубликаты, не забывая сбросить индексы
client_data = client_data.drop_duplicates().reset_index(drop=True)

### -----------------------------------------------------------------------------------------------------------------------------

Проверим результат:

### -----------------------------------------------------------------------------------------------------------------------------

In [650]:
print('Количество дублей:', client_data.duplicated().sum())

Количество дублей: 0


### Вывод

В данных было найдено 54 строки-дубликата. В связи с их небольшим числом они были удалены.

Банку стоит обратить внимание на процедуру заведения клиента в информационные системы, так как задвоение профилей может привести к значительным искажениям финансовых и статистических метрик, например, по целевой аудиториии банка.

## Обработка столбцов на предмет аномальных значений

### Столбец 'children'

### -----------------------------------------------------------------------------------------------------------------------------

Как мы выяснили выше, в столбце есть аномальные значения '-1' и '20':

### -----------------------------------------------------------------------------------------------------------------------------

In [651]:
client_data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

### -----------------------------------------------------------------------------------------------------------------------------

Определим количество строк с такими  значениями:

### -----------------------------------------------------------------------------------------------------------------------------

In [652]:
print("Количество строк со значением '-1':", client_data[client_data['children'] == -1]['children'].count())
print("Количество строк со значением '20':", client_data[client_data['children'] == 20]['children'].count())

Количество строк со значением '-1': 47
Количество строк со значением '20': 76


### -----------------------------------------------------------------------------------------------------------------------------

Небольшое количество строк со значением '-1' скорее всего свидетельствует о случайном характере ошибки при заполнении профиля клиента. Целесообразно заменить на 1.

### -----------------------------------------------------------------------------------------------------------------------------

In [653]:
# Заменяем '-1' на '1'
client_data['children'] = client_data['children'].replace(-1, 1)

### -----------------------------------------------------------------------------------------------------------------------------

Проверяем успешность замены:

### -----------------------------------------------------------------------------------------------------------------------------

In [654]:
client_data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

### -----------------------------------------------------------------------------------------------------------------------------

Теперь разберемся со значением '20'. В принципе такое количество детей возможно, но настораживает количество таких клиентов в наше выборке (76).
Посмотрим как такие клиенты распределены по возрасту, чтобы убедиться в том, что данные корректны. C учетом малой вероятности, что человек младше 38 лет может иметь 20 детей (если только они не приемные), разделим клиентов, имеющих 20 детей, по возрасту на две категории:

1. Менее 37 лет
2. От 38 лет и более

и посмотрим на их количественное соотношение:

### -----------------------------------------------------------------------------------------------------------------------------

In [655]:
# создаем функцию определяющую вероятность наличия 20 детей, в зависимости от возраста клиента (с защитой от ввода нечисловых значений)
def may_be_20(client_age):
    try:
        if client_age < 38:
            return 'невозможно'
        return 'возможно'
    except:
        print('Возраст необходимо указывать в числовом выражении')

# создаем новый столбец '20_possibility' где будет храниться вероятность наличия у клиента 20 детей
client_data['20_possibility'] = client_data['age'].apply(may_be_20)

### -----------------------------------------------------------------------------------------------------------------------------

Выведем сравнение в сводную таблицу

### -----------------------------------------------------------------------------------------------------------------------------

In [656]:
child_20 = client_data[(client_data['children'] == 20) & (client_data['20_possibility'])]
child_20_pivot = child_20.pivot_table(index = ['children'], columns = '20_possibility', values = 'age', aggfunc = 'count')
child_20_pivot

20_possibility,возможно,невозможно
children,,
20,46,30


### -----------------------------------------------------------------------------------------------------------------------------

Как мы видим - у нас большое число клиентов, в чьем возрасте объективно не может быть 20 детей. Соответственно, целесообразно считать такое значение ошибкой ввода. Изменяем '20' на '2' и удаляем столбец `20_possibility` за ненадобностью.

### -----------------------------------------------------------------------------------------------------------------------------

In [657]:
# Заменяем '20' на '2'
client_data['children'] = client_data['children'].replace(20, 2)
# Удаляем столбец '20_possibility'
del client_data['20_possibility']

### -----------------------------------------------------------------------------------------------------------------------------

Проверим значения столбца `children`

### -----------------------------------------------------------------------------------------------------------------------------

In [658]:
client_data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### -----------------------------------------------------------------------------------------------------------------------------

Столбец `children` в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'seniority'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [659]:
client_data['seniority'].describe()

count     21471.000000
mean      56821.423140
std      135010.270744
min      -18388.949901
25%       -2522.536607
50%        -989.271304
75%           0.000000
max      401755.400475
Name: seniority, dtype: float64

### -----------------------------------------------------------------------------------------------------------------------------

Проблемы:

* если, как утверждалось в описании, данные указаны в формате дней, то картина складывается странная, так как средний стаж, в таком случае, более 155 лет, а максимальный - более 1 100 лет. Нереально.
* отрицательные значения

Разберемся по порядку.

Посмотрим как распределены отрицательные значения стажа относительно типа занятости (столбец `position`)

### -----------------------------------------------------------------------------------------------------------------------------

In [660]:
# строим сводную таблицу по количеству отрицательных значений 'seniority' с группировкой по 'position'
negative_pivot = client_data[client_data['seniority'] < 0].pivot_table(index = 'position', values = 'seniority', aggfunc = 'count')
negative_pivot

,seniority
position,
в декрете,1
госслужащий,1312
компаньон,4577
предприниматель,1
сотрудник,10014
студент,1


### -----------------------------------------------------------------------------------------------------------------------------

Как мы видим, ранее высказанная нами гипотеза о том что отрицательные стажи "принадлежат" пенсионерам не подтвердилась. Подавляющая часть отрицательных значений стажа приходиться на "работающие" категории клиентов и, очевидно, являются обычным типом записи данных о стаже в данном банке. Таким образом имеет смысл взять эти значения по модулю и далее работать с ними в обычном режиме.

### -----------------------------------------------------------------------------------------------------------------------------

In [661]:
# берем значение 'seniority' по модулю
client_data['seniority'] = client_data['seniority'].abs()

# проверяем
client_data['seniority'].describe()

count     21471.000000
mean      60307.713617
std      133489.355354
min           0.000000
25%         620.736110
50%        1818.689386
75%        4794.911909
max      401755.400475
Name: seniority, dtype: float64

### -----------------------------------------------------------------------------------------------------------------------------

С отрицательными значениями разобрались. Теперь проработаем непосредственно величины `seniority`.

Имеет смысл предположить, что стаж все таки указан не формате дней, а, судя по величинам,в формате часов. Посчитаем среднее и максимальное значение (в формате лет), исходя из вышеуказанного предположения.

### -----------------------------------------------------------------------------------------------------------------------------

In [662]:
print('Среднее значение стажа, в формате лет:',int(60307.263611 / 24 / 365))
print('Максимальный стаж, в формате лет:',int(401755.000000 / 24 / 365))

Среднее значение стажа, в формате лет: 6
Максимальный стаж, в формате лет: 45


### -----------------------------------------------------------------------------------------------------------------------------

При допущении , что исходные данные стажа указаны в формате часов - картина выглядит гораздо более правдоподобно. Принимаем за истину и преобразуем данные столбца в формат лет и изменим тип на int. В том числе для повышения удобочитаемости.

### -----------------------------------------------------------------------------------------------------------------------------

In [663]:
# Преобразуем часы в годы и округлим значения до сотых
client_data['seniority'] = client_data['seniority'] / 24 / 365
client_data['seniority'] = client_data['seniority'].round(2)
client_data

,children,seniority,age,education,education_id,family_status,family_status_id,gender,position,debt,client_income,purpose
0,1,0.96,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,0.46,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,0.64,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,0.47,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,38.84,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,0.52,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21467,0,39.26,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,0.24,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21469,3,0.36,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### -----------------------------------------------------------------------------------------------------------------------------

Со столбцом 'seniority' закончили.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'age'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [664]:
client_data['age'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [665]:
client_data['age'].describe()

count    21471.000000
mean        43.279074
std         12.574291
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: age, dtype: float64

### -----------------------------------------------------------------------------------------------------------------------------

Проблем не замечено.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'education'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [666]:
client_data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

### -----------------------------------------------------------------------------------------------------------------------------

Значения столбца отличаются регистром. Стандартизируем методом `.str.lower()`

### -----------------------------------------------------------------------------------------------------------------------------

In [667]:
client_data['education'] = client_data['education'].str.lower()

# Проверяем
client_data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### -----------------------------------------------------------------------------------------------------------------------------

Столбец в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

In [668]:
# проверим на появление дублей после замены регистра
client_data.duplicated().sum()

17

In [669]:
new_dupl = client_data[client_data.duplicated(keep = False)].sort_values(by = ['age'])
new_dupl

,children,seniority,age,education,education_id,family_status,family_status_id,gender,position,debt,client_income,purpose
20248,1,0.0,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
8847,1,0.0,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
4214,0,0.0,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
6308,0,0.0,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
19515,0,0.0,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,свой автомобиль
3394,0,0.0,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,свой автомобиль
16121,0,0.0,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
19326,0,0.0,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
9739,0,0.0,46,среднее,1,гражданский брак,1,F,сотрудник,0,142594,покупка жилья для сдачи
14705,0,0.0,46,среднее,1,гражданский брак,1,F,сотрудник,0,142594,покупка жилья для сдачи


Посмотрим на рассредоточенность дублей по группам занятости и общий объем записей групп занятости:

In [670]:
new_dupl['position'].value_counts()

пенсионер    15
сотрудник    14
компаньон     4
Name: position, dtype: int64

In [671]:
client_data['position'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: position, dtype: int64

Учитывая малое число дубликатов, относительно общего размера выборок соответствующих групп занятости, целесообразно просто удалить их (не забыв сбросить индексы)

In [672]:
client_data = client_data.drop_duplicates().reset_index(drop=True)

Контрольная проверка наличия дубликатов:

In [673]:
client_data.duplicated().sum()

0

### Столбец 'education_id'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [674]:
client_data['education_id'].unique()

array([0, 1, 2, 3, 4])

### -----------------------------------------------------------------------------------------------------------------------------

Количество идентификаторов совпадает с количеством значений в столбце `education`.

Столбец в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'family_status'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [675]:
client_data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

### -----------------------------------------------------------------------------------------------------------------------------

Не все статусы в одном регистре. Исправим, приведя все значения к нижнему регистру.

### -----------------------------------------------------------------------------------------------------------------------------

In [676]:
client_data['family_status'] = client_data['family_status'].str.lower()

### -----------------------------------------------------------------------------------------------------------------------------

Проверим результат.

### -----------------------------------------------------------------------------------------------------------------------------

In [677]:
client_data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

### -----------------------------------------------------------------------------------------------------------------------------

Статусы стандартизированы.

Столбец в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'family_status_id'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [678]:
client_data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

### -----------------------------------------------------------------------------------------------------------------------------

Количество идентификаторов совпадает с количеством значений в столбце `family_status`.

Столбец в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'gender'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [679]:
client_data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

### -----------------------------------------------------------------------------------------------------------------------------

Обнаружено аномальное значение 'XNA'. Проверим массовость этого значения.

### -----------------------------------------------------------------------------------------------------------------------------

In [680]:
client_data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

### -----------------------------------------------------------------------------------------------------------------------------

'XNA' является единичным случаем. Можно пренебречь, учитывая что столбец `gender` в рамках задач проекта нам не понадобиться.

Столбец в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'position'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [681]:
client_data['position'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

### -----------------------------------------------------------------------------------------------------------------------------

Значения стандартизированы.

Столбец в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'debt'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [682]:
client_data['debt'].unique()

array([0, 1])

### -----------------------------------------------------------------------------------------------------------------------------

Два значения:

* '0' - нет задолженности
* '1' - есть задолженность

Все так как нам надо.

Столбец в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'client_income'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [683]:
# отобразим статистику с округлением до десятых
client_data['client_income'].describe().apply("{0:.1f}".format)

count      21454.0
mean      165319.6
std        98187.3
min        20667.0
25%       107623.0
50%       142594.0
75%       195820.2
max      2265604.0
Name: client_income, dtype: object

### -----------------------------------------------------------------------------------------------------------------------------

Обращает на себя внимание максимальный доход - свыше 2,2 млн. Даже в рублях внушительная сумма. Посмотрим на массовость такого явления.

### -----------------------------------------------------------------------------------------------------------------------------

In [684]:
# построим сводную таблицу по доходам свыше 2 млн. с группировкой по 'position'
mio_pivot = client_data[client_data['client_income'] > 2000000].pivot_table(index = 'position', values = 'client_income', aggfunc = 'count')
mio_pivot

,client_income
position,
компаньон,2


### -----------------------------------------------------------------------------------------------------------------------------

Похоже на правду. Совладельцы компаний вполне могут иметь такие доходы. И их количество невелико. Принимается.

Столбец в порядке.

### -----------------------------------------------------------------------------------------------------------------------------

### Столбец 'purpose'

### -----------------------------------------------------------------------------------------------------------------------------

Проанализируем:

### -----------------------------------------------------------------------------------------------------------------------------

In [685]:
client_data['purpose'].unique().tolist()

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

### -----------------------------------------------------------------------------------------------------------------------------

Количество целей кредита достаточно велико, но очевидна взаимосвязь по отдельным основным темам. С целью выяснения основных тем применим лемматизацию.

### -----------------------------------------------------------------------------------------------------------------------------

## Лемматизация

In [686]:
m = Mystem()

# создаем строку значений 'purpose' с разделителем ','
purpose_values = client_data['purpose'].tolist()
text = ','.join(purpose_values)

# лемматизируем текст и применяем счетчик
lemmas = m.lemmatize(text)
print(Counter(lemmas))

Counter({' ': 33570, ',': 21453, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


### -----------------------------------------------------------------------------------------------------------------------------

Из существительных, если отбросить не имеющих конкретики (например, 'покупка', 'операция' и т.п.), наиболее часто встречаемыми являются:

* 'недвижимость' - 6 353
* 'жилье' - 4 461
* 'автомобиль' - 4 308
* 'образование' - 4 014
* 'свадьба' - 2335

Остальные значения имеют гораздо меньший вес, поэтому в дальнейшем сосредоточимся на вышеперечисленных основных категориях. Несмотря на то, что логичным шагом выглядит объединение категорий 'жилье' и 'недвижимость', все же не будем этого делать, так как покупка коммерческой недвижимости является отдельной потребностью. Таким образом, наш список целей кредита будет состоять из 5 категорий: 'жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба'.

Заведем соответствующую переменную:

### -----------------------------------------------------------------------------------------------------------------------------

In [687]:
category_of_purpose = ['жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба']

### -----------------------------------------------------------------------------------------------------------------------------

Необходимо добавить в таблицу новый столбец с основными категориями. Но сначала напишем функцию, которую будем применять при добавлении столбца.

### -----------------------------------------------------------------------------------------------------------------------------

In [688]:
def main_category(text):
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        for category in category_of_purpose:
            if category in lemma:
                return category

### -----------------------------------------------------------------------------------------------------------------------------

Проверим работу функции:

### -----------------------------------------------------------------------------------------------------------------------------

In [689]:
print(main_category('на приобретение жилья'))

жилье


### -----------------------------------------------------------------------------------------------------------------------------

Работает. Добавим столбец `category_of_purpose` в таблицу и проверим результат.

### -----------------------------------------------------------------------------------------------------------------------------

In [690]:
client_data['category_of_purpose'] = client_data['purpose'].apply(main_category)
client_data

,children,seniority,age,education,education_id,family_status,family_status_id,gender,position,debt,client_income,purpose,category_of_purpose
0,1,0.96,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,0.46,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,0.64,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,0.47,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,38.84,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,0.52,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье
21450,0,39.26,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21451,1,0.24,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21452,3,0.36,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


### Вывод

Несмотря на достаточно большое число вариантов целей кредита, с помощью лемматизации нам удалось свести перечень до 5 значений ('жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'), что повысит как удобство дальнейших расчетов, так и удобочитаемость информации.

## Категоризация данных

### -----------------------------------------------------------------------------------------------------------------------------

В рамках поиска ответов на поставленные в рамках проекта вопросы, а именно:

- влияет ли семейное положение на факт погашения кредита в срок?
- влияет ли количество детей клиента на факт погашения кредита в срок?
- есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- как разные цели кредита влияют на его возврат в срок?


необходимо будет сегментировать (категоризировать) данные с целью удобства расчетов и наглядности результатов.

Приступим.

Распеределим клиентов по наличию детей по следующим категориям:

- 0 детей - 'без детей'
- 1 ребенок - 'c 1 ребенком'
- 2 ребенка - 'c 2 детьми'
- 3 и более детей - 'многодетный(-ая)'

Напишем функцию, которую применим при добавлении в таблицу столбца `child_status`

### -----------------------------------------------------------------------------------------------------------------------------

In [691]:
def child_func(count):
    if count == 0:
        return 'без детей'
    if count == 1:
        return 'c 1 ребенком'
    if count == 2:
        return 'c 2 детьми'
    return 'многодетный(-ая)'
        

### -----------------------------------------------------------------------------------------------------------------------------

Проверим функцию

### -----------------------------------------------------------------------------------------------------------------------------

In [692]:
print(child_func(0))

без детей


### -----------------------------------------------------------------------------------------------------------------------------

Работает корректно. Добавим в таблицу столбец 'child_status' с применением созданной функции.

### -----------------------------------------------------------------------------------------------------------------------------

In [693]:
client_data['child_status'] = client_data['children'].apply(child_func)

### -----------------------------------------------------------------------------------------------------------------------------

Проверим

### -----------------------------------------------------------------------------------------------------------------------------

In [694]:
client_data

,children,seniority,age,education,education_id,family_status,family_status_id,gender,position,debt,client_income,purpose,category_of_purpose,child_status
0,1,0.96,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,c 1 ребенком
1,1,0.46,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,c 1 ребенком
2,0,0.64,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,без детей
3,3,0.47,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетный(-ая)
4,0,38.84,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,без детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,0.52,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,c 1 ребенком
21450,0,39.26,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,без детей
21451,1,0.24,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,c 1 ребенком
21452,3,0.36,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,многодетный(-ая)


### -----------------------------------------------------------------------------------------------------------------------------

Категоризация по детям завершена.

Переходим к семейному положению.

Имеющиеся в таблице данные по категории семейного положения (столбец `family_status_id`) недостаточно "читабельны".

Сделаем следующее, в связи со смысловой идентичностью статусов `женат / замужем` и `гражданский брак`, целесообразно отнести их в одну категорию `в браке`, а `не женат / не замужем`, `в разводе` и `вдовец / вдова` к категории `не в браке`.

Напишем функцию, которую применим при добавлении в таблицу столбца `family_global_status`

### -----------------------------------------------------------------------------------------------------------------------------

In [695]:
def change_family_status(status):
    if status == 'женат / замужем' or status == 'гражданский брак':
        return 'в браке'
    return 'не в браке'

### -----------------------------------------------------------------------------------------------------------------------------

Проверим функцию

### -----------------------------------------------------------------------------------------------------------------------------

In [696]:
print(change_family_status('гражданский брак'))

в браке


### -----------------------------------------------------------------------------------------------------------------------------

Работает. Добавим в таблицу столбец `family_global_status` с применением функции

### -----------------------------------------------------------------------------------------------------------------------------

In [697]:
client_data['family_global_status'] = client_data['family_status'].apply(change_family_status)
client_data

,children,seniority,age,education,education_id,family_status,family_status_id,gender,position,debt,client_income,purpose,category_of_purpose,child_status,family_global_status
0,1,0.96,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,c 1 ребенком,в браке
1,1,0.46,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,c 1 ребенком,в браке
2,0,0.64,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,без детей,в браке
3,3,0.47,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетный(-ая),в браке
4,0,38.84,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,без детей,в браке
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,0.52,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,c 1 ребенком,в браке
21450,0,39.26,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,без детей,в браке
21451,1,0.24,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,c 1 ребенком,в браке
21452,3,0.36,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,многодетный(-ая),в браке


### -----------------------------------------------------------------------------------------------------------------------------

С семейным положением разобрались.

Для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" необходимо разбить клиентов на категории по уровню дохода. Еще раз оценим основные показатели столбца `client_income`

### -----------------------------------------------------------------------------------------------------------------------------

In [698]:
client_data['client_income'].describe().apply("{0:.1f}".format)

count      21454.0
mean      165319.6
std        98187.3
min        20667.0
25%       107623.0
50%       142594.0
75%       195820.2
max      2265604.0
Name: client_income, dtype: object

### -----------------------------------------------------------------------------------------------------------------------------

узнаем медиану

### -----------------------------------------------------------------------------------------------------------------------------

In [699]:
print(client_data['client_income'].median())

142594.0


### -----------------------------------------------------------------------------------------------------------------------------

Учитывая, что медиана равна чуть более чем 135 тыс.,по-видимому, рублей, а таже сегодняшние реалии по средним размерам заработной платы (судя по медиане будем исходить из Москвы, так как градацию клиентов по географии нам не предоставили), целесообразно разбить доходы на следующие категории:

- менее 70 тыс.руб - `низкий уровень`
- от 70 тыс.руб. до медианы - `средний уровень`
- от медианы до 300 тыс.руб. - `уровень выше среднего`
- свыше 300 тыс.руб. - `высокий уровень`

Создадим соответствующую функцию.

### -----------------------------------------------------------------------------------------------------------------------------

In [700]:
def income_category(value):
    if value < 70_000:
        return 'низкий уровень'
    elif value <= client_data['client_income'].median():
        return 'средний уровень'
    elif client_data['client_income'].median() < value < 350_000:
        return 'выше среднего'
    else:
        return 'высокий уровень'

In [701]:
# Проверяем работоспособность функции
print(income_category(150000))

выше среднего


### -----------------------------------------------------------------------------------------------------------------------------

Работает. Добавим в таблицу столбец `client_income_category` с использованием функции

### -----------------------------------------------------------------------------------------------------------------------------

In [702]:
client_data['client_income_category'] = client_data['client_income'].apply(income_category)
client_data

,children,seniority,age,education,education_id,family_status,family_status_id,gender,position,debt,client_income,purpose,category_of_purpose,child_status,family_global_status,client_income_category
0,1,0.96,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,c 1 ребенком,в браке,выше среднего
1,1,0.46,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,c 1 ребенком,в браке,средний уровень
2,0,0.64,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,без детей,в браке,выше среднего
3,3,0.47,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетный(-ая),в браке,выше среднего
4,0,38.84,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,без детей,в браке,выше среднего
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,0.52,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,c 1 ребенком,в браке,выше среднего
21450,0,39.26,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,без детей,в браке,выше среднего
21451,1,0.24,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,c 1 ребенком,в браке,средний уровень
21452,3,0.36,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,многодетный(-ая),в браке,выше среднего


### Вывод

Произвели категоризацию данных по количеству детей, семейному положению, доходу, что позволит нам наглядно решить задачи проекта. Категоризацию данных по цели кредита мы провеи ранее в процессе лемматизации.

Можно приступать к поиску ответов на задачи проекта.

## Шаг 3. Ответьте на вопросы

### Вопрос 1

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [706]:
pd.pivot_table(client_data, index='child_status', values='debt').sort_values(by = 'debt', ascending = False) 

,debt
child_status,
c 2 детьми,0.094925
c 1 ребенком,0.091658
многодетный(-ая),0.081579
без детей,0.075438


### Вывод 1

- ярко выраженной зависимости между наличием/количеством детей не обнаружено (от 7,5% до 9,5% клиентов имели задолженность,вне зависимости от наличия и количества детей)
- интересно, что многодетные клиенты, возвращают кредиты чаще чем клиенты с 1 или 2 детьми (8,1% против 9,1% и 9,5% соответственно)
- наблюдается несколько большая вероятность возврата кредита клиентами без детей, имеющих самый низкий процент невозврата (7,5%)

### Вопрос 2

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Добавим более детальную группировку(исходную) по семейному статусу

In [707]:
pd.pivot_table(client_data, index=['family_global_status','family_status'], values='debt').sort_values(by = 'debt', ascending = False) 

debt
family_global_status family_status                  
не в браке           не женат / не замужем  0.097509
в браке              гражданский брак       0.093471
                     женат / замужем        0.075452
не в браке           в разводе              0.071130
                     вдовец / вдова         0.065693

### Вывод 2

- категория клиентов, не состоящих в браке, склонна не возвращать кредиты чаще чем клиенты, состоящие в браке (8,5% против 7,9% соответственно). То есть зависимость есть, но слабо выраженная.
- но в то же время, если более детально проанализировать результаты, то видно, что внутри категории `не в браке` распределение вероятности невозврата кредита крайне неравомерно. Вдовцы и вдовы значительно реже не возвращают кредит, чем просто не женатые/ не замужние клиенты (6,5% против 9,7% соответственно) и, вообще, являются наиболее благонадежной категорией клиентов. В то время как не женатые/ не замужние возглавляют антирейтинг.   
- вообще, глядя на более детальную группировку по семейному положению, приходиться признать, что клиенты никогда не состоявшие в официальном браке (категории `не женат / не замужем` и `гражданский брак`) менее благонадежны на предмет отсутствия задолженностей по кредиту.

### Вопрос 3

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [708]:
pd.pivot_table(client_data, index='client_income_category', values='debt').sort_values(by = 'debt', ascending = False) 

,debt
client_income_category,
средний уровень,0.086349
выше среднего,0.079573
низкий уровень,0.068521
высокий уровень,0.064028


### Вывод 3

- наблюдается зависимость между процентом невозвратов кредита в срок и уровнем дохода
- самыми дисциплинированными и надежными клиентами являются, что ожидаемо, клиенты с высоким уровнем дохода (всего 6,4% невозвратов)
- интересным фактом является, то что клиенты с низким уровнем дохода значительно лучше отдают кредиты, чем люди с достатком средним и выше среднего (7,5% против 8,2% и 8,4% соответственно). Видимо такие клиенты отличаются большей самодисциплиной и ответственостью в ведении собственного бюджета.

### Вопрос 4

- Как разные цели кредита влияют на его возврат в срок?

In [709]:
pd.pivot_table(client_data, index='category_of_purpose', values='debt').sort_values(by = 'debt', ascending = False)

,debt
category_of_purpose,
автомобиль,0.093590
образование,0.092200
свадьба,0.080034
недвижимость,0.074634
жилье,0.069058


### Вывод 4

- есть достаточно четкая зависимость между целью кредита и его возврат клиентом в срок
- наиболее привлекательными для банка являются категории кредита с целью приобретения любой недвижимости, как жилой (6,9% невозвратов), так и коммерческой (7,4% невозвратов)
- рискованными для банка являются автокредитование и целевые образовательные кредиты (9,3% и 9,2% невозвратов соответственно)

## Шаг 4. Общий вывод

Полученные в ходе проекта результаты позволяют утверждать о существовании зависимостей по каждому из вопросов проекта. Но, обязательно необходимо учитывать, что разброс значений в 1-2 процента (а инода и того меньше) не позволяет однозначно рекомендовать результаты как руководство к действию.


Исходя из результатов проекта, портрет идеального заемщика выглядит так:


- женат/замужем, но не состоит в гражданском браке
- без детей
- имеет высокий уровень дохода (от 300 тыс.руб.)
- берет кредит для приобретения жилья или коммерческой недвижимости


Допполнительная информация:

Несмотря на полученные результаты, необходима дальнейшая, более детальная, проработка затронутых в проекте вопросов.

Стоит, по возможности, предпринять следующие шаги:
- расширить выборку (если возможно)
- получить недостающую информацию по текущей базе (например, регионы проживания для более детальной категоризации)
- свести к минимуму ошибки при заведении профиля клиента в информационных системах банка
- обеспечить грамотное взаимодействие информационных ресурсов банка для исключения возможностей задвоение профилей и т.п.
- рассмотреть возможность смены форматов хранения типов данных, для облегчения их последующей интерпретации и обработки (например, стаж)

Спасибо за внимание.